# Email processing

In [96]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import re
import pprint
import string

In [85]:
# download the emails as a .csv file

In [133]:
# open CSV
emails = pd.read_csv('emails.csv', nrows=2500)

In [134]:
# define the headers we'll catch
headers = [email.split(': ', 1)[0] for email in emails.iloc[0]['message'].split('\n')[:15]]

# create new column with each metadata field
f = lambda email: re.findall(rf'{header}: (.*)\n', email['message'])[0]
for header in headers:
    emails[header] = emails.apply(f, axis=1)

# new column with email content
f = lambda email: ''.join(re.findall('\n\n(.*)', email['message']))
emails['Body'] = emails.apply(f, axis=1)

# the "message" col is now useless
emails.drop(['message'], axis=1, inplace=True)

In [135]:
emails.head(2)

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Body
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...


## How to insert data into MongoDB?

In [136]:
# set up client
mongo = MongoClient()

# create/get database and collection
db = mongo['simplon']
db["enron"].drop() # in case we reimport later
col = db['enron']

# insert every email to mongodb
col.insert_many(emails.to_dict('records'))

In [137]:
# check
#col.find_one()

# Exercise 1 :


# Create a DataFrame showing the top 3 words used for each user (all of their emails aggregated)

#### Create a "Bag of Words" of each email body, in a new column

In [148]:
def get_words(email):
    """Return cleaned (html tags, punctuations, lowercase) text as a space-separated string"""
    
    # switch text to lowercase
    email = email.lower()
    
    # "subtract" embedded HTML tags (but keep the content)
    email = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", " ", email)
    email = re.sub(r"(?s)<!--(.*?)-->[\n]?", " ", email)
    email = re.sub(r"(?s)<.*?>", " ", email)
    email = re.sub(r"&nbsp;", " ", email)
    email = re.sub(r" +", " ", email)
    
    # remove punctuation
    email = re.sub(rf"[{string.punctuation}]", "", email)
    
    # get words
    words = re.findall("\w+", email)
    
    return ' '.join(words)

# Create new column
emails['Body_Words'] = emails["Body"].apply(get_words)

# Display
emails.head(1)

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Body,Body_Words
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,here is our forecast


#### Aggregate the BoWs by expeditor ("From")

In [149]:
emails.groupby(['From'])["Body_Words"]

In [150]:
grouped_df = pd.DataFrame(emails.groupby(['From'])['Body_Words'].apply(list))
grouped_df.head()

,Body_Words
From,
1.11913372.-2@multexinvestornetwork.com,[in todays daily update youll find free report...
adrianne.engler@enron.com,"[hi phillip and mike, phillipmike, phillipmike..."
al.pollard@newpower.com,[what is going on down the street curious to h...
alyse.herasimchuk@enron.com,[dear associate analyst committeethe following...
anchordesk_daily@anchordesk.zdlists.com,[david courseymeet the dark side of windows xp...


#### Show the 3 most common words

In [151]:
def top_3_bow(bow_array):
    """From a list of words, return a dict with top3 most frequent words and the number of occurences"""
    words = bow_array
    bow = {word:words.count(word) for word in words}
    top3_bow = {w:c for w,c in sorted(bow.items(), key=lambda item: item[1], reverse=True)[:3]}
    return top3_bow

In [152]:
grouped_df['Body_Bow_top3'] = grouped_df['Body_Words'].apply(top_3_bow)
grouped_df.head(3)

,Body_Words,Body_Bow_top3
From,,
1.11913372.-2@multexinvestornetwork.com,[in todays daily update youll find free report...,{'in todays daily update youll find free repor...
adrianne.engler@enron.com,"[hi phillip and mike, phillipmike, phillipmike...","{'phillipmike': 2, 'hi phillip and mike': 1, '..."
al.pollard@newpower.com,[what is going on down the street curious to h...,{'what is going on down the street curious to ...


# Exercise II :

# Apply TF-IDF to the dataset

#### (optional) Manually write a function that computes TF-IDF

```python
def occurs(word, doc):
    split_doc = doc.split()
    occs = [1 * (word == w.lower()) for w in split_doc]
    return sum(occs)

def idf(word, docs_list):
    N = len(docs_list)
    occs = [ 1 * (word in doc)  for doc in docs_list]
    return log(N / sum(presences))

def naive_tfidf(word, docs_list, curr_doc_idx):
    tf = occurs(word, docs_list[curr_doc_idx])
    idf = idf(word, docs_list)
    return tf*idf
```

#### Using SciKit Learn

In [154]:
# reminder 
emails.head(1)

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Body,Body_Words
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,here is our forecast


In [155]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define the corpus as the collection of emails, regardless of expeditor
corpus = emails["Body_Words"] # for now we work on the dataset made of individual emails (not aggregated)

# instantiate a vectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words='english')

# fit then get a transformed object (tfidf_matric = a matrix of tfidf scores)
tfidf_matrix = vectorizer.fit_transform(corpus)

# name the list of words (aka "features")
feature_names = vectorizer.get_feature_names()

In [157]:
# visualize the result
tfidf_matrix.toarray()
# It is a sparse matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [158]:
# Visualize the result as a dataframe (using toarray()) instead:
pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

,00,000,000000000009659,000000000021442,000000000067320,001mmbtu,005,00may,01,010,...,zdnetget,zdnetnew,zdnets,zdnetthrill,zero,zip,zone,zoning,zoya,zub
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [172]:
X = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

def getnwords(listofscores, n=5):
    dic = dict(zip(feature_names, listofscores))
    dic = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1])[-n:] if v > 0.1 }
    return dic.keys()

charac_words_df = X[feature_names].apply(getnwords, axis=1)
charac_words_df.name = "tf-idf top"
charac_words_df

0                                              (forecast)
1                  (try, takes, business, meetings, trip)
2                                 (way, successful, test)
3                  (send, schedule, level, randy, salary)
4                            (tuesday, lets, shoot, 1145)
                              ...                        
2495                  (width, add, rent, formula, column)
2496                                        (lucyphillip)
2497                                           (approval)
2498    (attached, spreadsheet, containing, referenced...
2499           (lucy, discuss, later, progress, creating)
Name: tf-idf top, Length: 2500, dtype: object

#### Add these results to the initial DataFrame (list of emails)

In [173]:
# reminder

print("'emails' DataFrame")
emails.head(1)

'emails' DataFrame


,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Body,Body_Words
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,here is our forecast


In [174]:
charac_words_df.head(1)

0    (forecast)
Name: tf-idf top, dtype: object

In [176]:
concat_df = pd.concat([emails, charac_words_df], axis=1)
concat_df[["Body", "tf-idf top"]].head()

,Body,tf-idf top
0,Here is our forecast,(forecast)
1,Traveling to have a business meeting takes the...,"(try, takes, business, meetings, trip)"
2,test successful. way to go!!!,"(way, successful, test)"
3,"Randy, Can you send me a schedule of the salar...","(send, schedule, level, randy, salary)"
4,Let's shoot for Tuesday at 11:45.,"(tuesday, lets, shoot, 1145)"


Safety (human) check:

In [177]:
concat_df["Body"].iloc[1]

"Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time."

In [178]:
concat_df["Body"].iloc[3]

'Randy, Can you send me a schedule of the salary and level of everyone in the Phillip'

Looks kinda relevant ! :)